In [5]:
import pygame
import speech_recognition as sr

# Initialize Pygame
pygame.init()

# Screen dimensions
WIDTH, HEIGHT = 1000, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Lampu Gedung Voice Control dengan Manajemen Daya dan Peringatan")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
YELLOW = (255, 255, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
PURPLE = (128, 0, 128)
ORANGE = (255, 165, 0)
GRAY = (169, 169, 169)
SKY_BLUE = (135, 206, 235)
DARK_GRAY = (50, 50, 50)

# Lamp state for each building and its light color and power consumption (in watts)
lamp_states = {
    "gedung 5": False,
    "gedung 7": False,
    "gedung 8": False,
    "gedung 9": False,
    "kantin": False,
    "asrama": False
}

lamp_colors = {
    "gedung 5": YELLOW,
    "gedung 7": RED,
    "gedung 8": GREEN,
    "gedung 9": BLUE,
    "kantin": ORANGE,
    "asrama": PURPLE
}

# Power consumption for each building (in watts)
lamp_power = {
    "gedung 5": 100,
    "gedung 7": 200,
    "gedung 8": 300,
    "gedung 9": 400,
    "kantin": 500,
    "asrama": 600
}

# Power management
total_power_capacity = 1000  # Max power capacity for the electric tower
current_power_usage = 0      # Current power usage
warning_message = ""         # Variable to store warning message

# Function to draw a building with windows, doors, and light
def draw_building(x, y, width, height, is_on, color, name, wattage):
    # Draw main building structure
    pygame.draw.rect(screen, DARK_GRAY if is_on else GRAY, (x, y - height, width, height))
    
    # Draw windows
    window_color = color if is_on else WHITE
    window_width = width // 4
    window_height = height // 5
    for i in range(2):
        for j in range(3):
            pygame.draw.rect(screen, window_color, (x + 10 + i * (window_width + 10), y - height + 20 + j * (window_height + 10), window_width, window_height))

    # Draw door
    pygame.draw.rect(screen, BLACK, (x + width // 3, y - 30, width // 3, 30))

    # Draw building name and wattage
    font = pygame.font.SysFont(None, 24)
    label = font.render(f"{name.upper()} ({wattage}W)", True, BLACK)
    screen.blit(label, (x + (width // 2 - label.get_width() // 2), y + 10))

# Function to draw power tower (electricity tower)
def draw_power_tower():
    pygame.draw.rect(screen, DARK_GRAY, (50, 100, 50, 300))  # Tower pole
    pygame.draw.circle(screen, BLACK, (75, 80), 50)  # Power capacity circle
    
    # Show power usage text
    font = pygame.font.SysFont(None, 36)
    text = font.render(f'Power: {current_power_usage}/{total_power_capacity}W', True, WHITE)
    screen.blit(text, (10, 50))

# Function to draw warning message
def draw_warning_message():
    global warning_message
    if warning_message:
        font = pygame.font.SysFont(None, 48)
        warning_label = font.render(warning_message, True, RED)
        screen.blit(warning_label, (WIDTH // 2 - warning_label.get_width() // 2, HEIGHT // 2 - 50))

# Function to draw buildings with more detail
def draw_buildings():
    screen.fill(SKY_BLUE)  # Background color (sky)
    
    # Draw power tower
    draw_power_tower()

    # Coordinates for buildings
    building_positions = {
        "gedung 5": (150, 500),
        "gedung 7": (300, 500),
        "gedung 8": (450, 500),
        "gedung 9": (600, 500),
        "kantin": (750, 500),
        "asrama": (900, 500)
    }

    # Draw each building
    for building, (x, y) in building_positions.items():
        building_width = 100
        building_height = 200
        draw_building(x, y, building_width, building_height, lamp_states[building], lamp_colors[building], building, lamp_power[building])

    # Draw warning message if any
    draw_warning_message()

    pygame.display.update()

# Initialize recognizer
recognizer = sr.Recognizer()

# Function to recognize speech with mic sensitivity adjustment
def recognize_speech():
    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source)  # Kalibrasi terhadap kebisingan sekitar
        print("Dengarkan...")
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio, language="id-ID")
            print(f"Kamu bilang: {text}")
            return text.lower()
        except sr.UnknownValueError:
            print("Maaf, tidak bisa mengenali suara.")
            return None
        except sr.RequestError:
            print("Tidak bisa menghubungi Google Speech Recognition.")
            return None

# Function to manage power and toggle lights
def toggle_lights(command):
    global current_power_usage, warning_message
    warning_message = ""  # Clear any previous warning
    for building in lamp_states:
        if building in command:
            if lamp_states[building]:  # If the building is already ON, turn it OFF
                lamp_states[building] = False
                current_power_usage -= lamp_power[building]
            else:  # If the building is OFF, check if we have enough power
                if current_power_usage + lamp_power[building] <= total_power_capacity:
                    lamp_states[building] = True
                    current_power_usage += lamp_power[building]
                else:
                    warning_message = f"Daya tidak cukup untuk menyalakan {building.upper()}!"
                    print(warning_message)
    
    # Optional command to turn off all lights
    if "matikan semua lampu" in command:
        for building in lamp_states:
            if lamp_states[building]:
                current_power_usage -= lamp_power[building]
            lamp_states[building] = False

# Main loop
running = True
while running:
    draw_buildings()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        # Check for key press event once (not continuously in loop)
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:  # Listen when 'r' key is pressed
                command = recognize_speech()
                if command:
                    toggle_lights(command)

    pygame.time.delay(100)

pygame.quit()

Dengarkan...
Kamu bilang: nyalakan lampu j5
Dengarkan...
Kamu bilang: lampu biru
Dengarkan...
Kamu bilang: lampu gedung 5
Dengarkan...
Kamu bilang: nyalakan lampu jam 09
Dengarkan...
Kamu bilang: lampu gedung 9
Dengarkan...
Kamu bilang: lampu kantil
Dengarkan...
Kamu bilang: lampu samping
Dengarkan...
Maaf, tidak bisa mengenali suara.
Dengarkan...
Kamu bilang: lampu pantun
Dengarkan...
Kamu bilang: lampu kantin
Dengarkan...
Kamu bilang: lampu ring 7
Dengarkan...
Kamu bilang: gedung 7
Daya tidak cukup untuk menyalakan GEDUNG 7!
